# 基于卷积神经网络实现猫狗识别

下载数据集到本地，[链接](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data),下载方法可见 README.md

### 数据导入与预处理

In [1]:
import cv2 
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.preprocessing import image
from PIL import Image
from IPython.display import display
from keras.utils import np_utils
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
C:\Users\yeyiy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 定义调用模块

### 制作软连接模块

In [13]:
TRAIN_DATA_DIR = 'C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe'
TEST_DATA_DIR = 'C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe'

def make_symlink():

    image_dogs = []
    image_cats = []
    label_dogs = []
    label_cats = []
    test_images = []
    
    if not os.path.exists('newTrain'):
        os.mkdir('newTrain')
        os.mkdir('newTrain/cats')
        os.mkdir('newTrain/dogs')  
    else:
        for filename in os.listdir('newTrain/cats'):
            os.remove('newTrain/cats/'+ filename)
        for filename in os.listdir('newTrain/dogs'):
            os.remove('newTrain/dogs/'+ filename)
        
    if not os.path.exists('newTest'):
        os.mkdir('newTest')
    else:
        if os.path.exists('newTest/test'):
            for filename in os.listdir('newTest/test'):
                os.remove('newTest/test/'+ filename)
            os.rmdir('newTest/test/')
    
    
    for img in tqdm(os.listdir(TRAIN_DATA_DIR)):
        images = os.path.join(img)
#         display(Image.open(TRAIN_DATA_DIR+'/'+img))
        print(images)
        labels = img.split('.')[0]
        if labels == 'cat':
            image_cats.append(images)
            label_cats.append(0)
        else:
            image_dogs.append(images)
            label_dogs.append(1)
            
    print("image_cats:", image_cats)
    print("label_cats:", label_cats)
    print("label_dogs:", label_dogs)
            
    for img in tqdm(os.listdir(TEST_DATA_DIR)):
        images = os.path.join(img)
        test_images.append(images)
            
    for filename in image_cats:
        os.symlink(TRAIN_DATA_DIR + '/'+filename, 'newTrain/cats/'+filename)
        
    for filename in image_dogs:
        os.symlink(TRAIN_DATA_DIR + '/'+filename, 'newTrain/dogs/'+filename)
    
    os.symlink(TEST_DATA_DIR+'/', 'newTest/test')
    
    return None
    

In [14]:
make_symlink()

  0%|          | 0/15 [00:00<?, ?it/s]

cat.11.jpg
cat.16.jpg
cat.17.jpg
cat.22.jpg
cat.23.jpg
cat.3.jpg
dog.656.jpg
dog.7884.jpg
dog.7885.jpg
dog.7886.jpg
dog.7887.jpg
dog.7890.jpg
dog.7891.jpg
dog.7892.jpg
dog.7893.jpg


100%|██████████| 15/15 [00:00<00:00, 234.50it/s]


image_cats: ['cat.11.jpg', 'cat.16.jpg', 'cat.17.jpg', 'cat.22.jpg', 'cat.23.jpg', 'cat.3.jpg']
label_cats: [0, 0, 0, 0, 0, 0]
label_dogs: [1, 1, 1, 1, 1, 1, 1, 1, 1]


100%|██████████| 15/15 [00:00<?, ?it/s]


OSError: symbolic link privilege not held

### 生成器输出模块

In [4]:
def make_generator(image_size):

#     input_images = []
    
#     for i_img in i_images:
# #       images = cv2.imread(images, cv2.IMREAD_COLOR)
#         images = image.load_img(i_img, target_size=(1,image_size,image_size, 3))
#         images = image.img_to_array(images)
#         input_images.append(images)
#     input_images = np.array(input_images)
    
#     # one hot eencoding
#     input_labels = np_utils.to_categorical(i_labels)
    
    
    # 图像预处理
    datagen = image.ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True)
    
    train_batch = datagen.flow_from_directory('newTrain', target_size=(image_size,image_size),
                                 batch_size=64,shuffle=False,seed=10)
    
    test_batch = datagen.flow_from_directory('newTest', target_size=(image_size, image_size),
                                 batch_size=64, shuffle=False,class_mode=None, seed=10)
    
    return train_batch, test_batch



### 结果可视化模块

In [6]:
def show_history(t_history,train,valid):
    plt.plot(t_history.history[train])
    plt.plot(t_history.history[valid])
    plt.title('Xception Model '+ train +' History')
    plt.y_label(train)
    plt.x_label('Epoch')
    plt.legend(['train '+train, 'valid '+valid], loc='upper left')
    plt.show()
    

# 建立模型
使用预训练过的模型，使用迁移学习的思想进行模型的建立，并试图进行模型融合训练，使用到的模型有
- Inception v3
- InceptionResNetV2
- Xception

## Xception模型

In [11]:
from keras.applications import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras.optimizers import SGD
from IPython.display import SVG
from keras.layers import *
import h5py as h5py

In [50]:
model_xception = Xception(include_top=False, weights='imagenet', input_shape=(299,299,3))

for layer in model_xception.layers:
    layer.trainable = False
    
x = GlobalAveragePooling2D()(model_xception.output)

model1 = Flatten(name='flatten')(model_xception.output)
model1 = Dense(4096, activation='relu', name='fc1')(model1)
model1 = Dense(4096, activation='relu', name='fc2')(model1)
model1 = Dropout(0.5)(model1)
model1 = Dense(2, activation='softmax', name='prediction')(model1)

model_xception_pred = Model(model_xception.input, model1, name='xception')


ValueError: Input 0 is incompatible with layer flatten: expected min_ndim=3, found ndim=2

- 选择损失函数(loss)为：categorical_crossentropy  
- 选择优化器(optimizer)为：Nadam() 
- 选择评价函数为：accuracy

In [16]:
model_xception_pred.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

### 训练模型

In [49]:
epochs = 10
image_size = 299

make_symlink()
# train_generator, test_generator = make_generator(image_size)
# xception_history = model_xception_pred.fit_generator(train_generator, epochs=epochs, verbose=1)

100%|██████████| 12500/12500 [00:00<00:00, 578008.07it/s]


Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[204800,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: fc1_1/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=7628148, _device="/job:localhost/replica:0/task:0/device:GPU:0"](fc1_1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'fc1_1/random_uniform/RandomUniform', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
 

### 模型可视化

In [ ]:
SVG(model_to_dot(model_xception_pred, show_shapes=True).create(prog="dot", format='svg'))

### 执行结果可视化

In [ ]:
# accuracy 可视化
show_history(xception_history, 'acc', 'val_acc')

In [ ]:
# loss 可视化
show_history(xception_history, 'loss', 'val_loss')

## 建立混合模型

### Xception 模型

In [ ]:
model_xception = Xception(include_top=False, weights='imagenet', input_shape=(299,299,3))
x = GlobalAveragePooling2D()(model_xception.output)
xception_model = Model(model_xception.input, x)

xception_train = xception_model.predict_generate(train_generator)
xception_test = xception_model.predict_generator(test_generator)

# 将特征向量写入数据文件
with h5py.File("gap_Xception.h5") as h:
    h.create_dataset("train", data=xception_train)
    h.create_dataset("test", data=xception_test)
    h.create_dataset("t_label", data=train_generator.classes)


### Inception v3 模型

In [ ]:
model_inception_v3 = InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3))
y = GlobalAveragePooling2D()(model_inception_v3.output)
inceptionV3_model = Model(model_inception_v3.input, y)

inceptionV3_train = inceptionV3_model.predict_generate(train_generator)
inceptionV3_test = inceptionV3_model.predict_generator(test_generator)

# 将特征向量写入数据文件
with h5py.File("gap_InceptionV3.h5") as h:
    h.create_dataset("train", data=inceptionV3_train)
    h.create_dataset("test", data=inceptionV3_test)
    h.create_dataset("t_label", data=train_generator.classes)

### InceptionResNetV2 模型

In [ ]:
model_inceptionresnetv2 = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(299,299,3))
z = GlobalAveragePooling2D()(model_inceptionresnetv2.output)
inceptionresnetv2_model = Model(model_inceptionresnetv2.input, y)

inceptionresnetv2_train = inceptionresnetv2_model.predict_generate(train_generator)
inceptionresnetv2_test = inceptionresnetv2_model.predict_generator(test_generator)

# 将特征向量写入数据文件
with h5py.File("gap_InceptionResNetV2.h5") as h:
    h.create_dataset("train", data=inceptionresnetv2_train)
    h.create_dataset("test", data=inceptionresnetv2_test)
    h.create_dataset("t_label", data=train_generator.classes)

### 从文件中读取特征向量

In [ ]:
train_images = []
train_labels = []
valid_images = []
valid_labels = []
test_data = []

h5File = ["gap_Xception.h5", "gap_InceptionV3.h5", "gap_InceptionResNetV2.h5"]
for filename in h5File:
    for h5py.File(filename, 'r') as h:
        train_images.append(np.array(h['train']))
        train_labels.append(np.array(h['t_label']))
        test_data.append(np.array(h['test']))
        
# 将list竖直拼接
train_images = np.concatenate(train_images, axis=1)
# train_labels = np.concatenate(train_labels, axis=1)
test_data = np.concatenate(test_data, axis=1)

train_images, train_labels = shuffle(train_images, train_labels)


In [ ]:
train_input = Input(train_images.shape[1:])

model = Dropout(0.5)(train_input)
model = Dense(1, activation='sigmoid')(model)

new_model = Model(train_input, model)

In [ ]:
new_model.compile(loss='categorical_crossentropy', optimizer=Nadam(), metrics=['accuracy'])

In [ ]:
new_model.fit(train_images, train_labels, batch_size=128, epochs=10, verbose=2, validation_split=0.2)

## 测试

In [ ]:
test_pred = new_model.predict(test_data, verbose=1)

In [ ]:
test_pred = test_pred.clip(min=0.005, max=0.995)

file = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(TEST_DATA_DIR, (299, 299),
                                        shuffle=False, batch_size=16, class_mode=None)

for i, filename in enumerate(os.listdir('newTest')):
    index = int(filename.split('.')[0])
    file.set_value(index-1, 'label', test_pred[i])
    
file.to_csv('pred.csv', index=None)
file.head(10)
